In [525]:
# repo root
%cd /work

/work


In [276]:
!ls 

LICENSE    adapters  examples  paper	       resources  src
README.md  docker    out       pyproject.toml  scripts	  tests


In [277]:
#!chmod +x resources/gene_id_maps/build_entrez_id_map.sh
#! ./resources/gene_id_maps/build_entrez_id_map.sh

In [278]:
#!chmod +x resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh
#! ./resources/gene_id_maps/build_ensembl_to_symbol_from_gtf.sh --download

In [5]:
# Fetch from UCSC Xena
!python paper/scripts/fig2_fetch_inputs.py

[fetch_inputs] OK
  /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/mc3.v0.2.8.PUBLIC.xena.gz
  /work/paper/source_data/PANCAN_TP53_v1/raw/TCGA_phenotype_dense.tsv.gz


In [5]:
# Extract TP53 mutations (mc3.v0.2.8.PUBLIC.xena.gz)
# Create barcode→cancer from phenotype 
# Output derived/groups/{CANCER}.groups.tsv　for each cancer type

!python paper/scripts/fig2_make_groups.py

[make_groups] sample_type_id raw unique (head): ['07', '06', '05', '03', '02', '01', '11', '']
[make_groups] sample_type_id norm counts: {'1': 10593, '11': 1475, '6': 396, '3': 200, '': 72, '2': 56, '5': 11, '7': 1}
[make_groups] filtered to primary-like sample_type_id in {1,3}: 10793
[make_groups] phenotype mapped samples: 10793
[make_groups] cancers: ['ACC', 'BLCA', 'BRCA', 'CESC', 'CHOL', 'COAD', 'DLBC', 'ESCA', 'GBM', 'HNSC', 'KICH', 'KIRC', 'KIRP', 'LAML', 'LGG', 'LIHC', 'LUAD', 'LUSC', 'MESO', 'OV', 'PAAD', 'PCPG', 'PRAD', 'READ', 'SARC', 'SKCM', 'STAD', 'TGCT', 'THCA', 'THYM', 'UCEC', 'UCS', 'UVM']
[make_groups] MC3 TP53 rows: 3729
[make_groups] MC3 TP53 protein-altering rows (pre-FILTER): 3620
[make_groups] FILTER column not found; not applying FILTER
[make_groups] WARNING: LAML has n_mut=0, n_wt=200
[make_groups] WARNING: UVM has n_mut=0, n_wt=80
[make_groups] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/PANCAN.groups.tsv
  cancers: 33
  samples (mapped): 

In [3]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/sample_cards/*
!python paper/scripts/fig2_make_sample_cards.py

[make_sample_cards] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/sample_cards (33 cancers × 2 cards)


In [4]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.note.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "note",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [5]:
!cat /work/paper/source_data/PANCAN_TP53_v1/sample_cards/HNSC.hard.sample_card.json

{
  "comparison": "TP53_mut_vs_TP53_wt",
  "condition": "HNSC",
  "extra": {
    "audit_tau": 0.8,
    "benchmark_id": "PANCAN_TP53_v1",
    "context_gate_mode": "hard",
    "gene_id_map_tsv": "resources/gene_id_maps/id_map.tsv.gz",
    "goal": "context-conditioned pathway claim selection with calibrated abstention"
  },
  "k_claims": 100,
  "notes": "TCGA Pan-Cancer; groups defined from MC3 TP53 PASS mutations.",
  "perturbation": "genotype",
  "tissue": "tumor"
}


In [10]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
  !Rscript paper/scripts/fig2_deg_rank.R $c


[deg_rank] inputs
  cancer: HNSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/HNSC.groups.tsv 
[deg_rank] matched samples: 520  / expr= 11069  / groups= 528 
[deg_rank] QC: dropping zero-variance genes: 269 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  n_genes: 20262 
[deg_rank] inputs
  cancer: LUAD 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/groups/LUAD.groups.tsv 
[deg_rank] matched samples: 515  / expr= 11069  / groups= 520 
[deg_rank] QC: dropping zero-variance genes: 338 
[deg_rank] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/LUAD.deg_ranking.tsv 
  n_genes: 20193 
[deg_rank] inputs
  cancer: LUSC 
  expr: /work/paper/source_data/PANCAN_TP53_v1/raw/expression.xena.gz 
  groups: /work/paper/source_data/PANCAN_TP53_v1/derived/gr

In [11]:
!Rscript paper/scripts/fig2_fgsea_to_evidence_table.R HNSC


[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
 [100%] Downloaded 33730584 bytes...
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 


In [601]:
# Fig2 run
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours,context_swap \
  --k-claims 50 \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours,context_swap
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 10
[run_fig2] (1/10) HNSC ours gate=hard tau=0.20
[run_fig2] (2/10) HNSC context_swap gate=hard tau=0.20
[run_fig2] (3/10) HNSC ours gate=hard tau=0.40
[run_fig2] (4/10) HNSC context_swap gate=hard tau=0.40
[run_fig2] (5/10) HNSC ours gate=hard tau=0.60
[run_fig2] (6/10) HNSC context_swap gate=hard tau=0.60
[run_fig2] (7/10) HNSC ours gate=hard tau=0.80
[run_fig2] (8/10) HNSC context_swap gate=hard tau=0.80
[run_fig2] (9/10) HNSC ours gate=hard tau=0.90
[run_fig2] (10/10) HNSC context_swap gate=hard tau=0.90
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [602]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC

context_swap  ours


In [603]:
import pandas as pd, json, glob
base=glob.glob("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20")[0]
p=base+"/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
print("context_review_mode unique:", sorted(set(df.get("context_review_mode", pd.Series(dtype=str)).astype(str))))
print("context_status counts:\n", df.get("context_status", pd.Series(dtype=str)).astype(str).value_counts(dropna=False).head(10))
print("first row context_notes:", df.get("context_notes", pd.Series(dtype=str)).astype(str).iloc[0] if len(df) else "")
m=json.load(open(base+"/run_meta.json"))
print("sample in meta:", m.get("inputs",{}).get("sample",{}))

context_review_mode unique: ['proxy']
context_status counts:
 context_status
PASS       40
ABSTAIN     8
FAIL        2
Name: count, dtype: int64
first row context_notes: proxy_context_v2: u=0.597 p_fail=0.050 p_warn=0.200 key=['condition', 'tissue', 'perturbation', 'comparison'] ctx_sig=e6cee1e46cac
sample in meta: {'condition': 'HNSC', 'tissue': 'tumor', 'perturbation': 'genotype', 'comparison': 'TP53_mut_vs_TP53_wt', 'species': ''}


In [604]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.original.json
claims.proposed.tsv	    report.jsonl	  sample_card.resolved.json
distilled.tsv		    report.md		  sample_card.tau.json
distilled.with_modules.tsv  risk_coverage.tsv	  term_gene_edges.tsv
evidence.normalized.tsv     run_meta.json	  term_modules.tsv
manifest.json		    run_meta.runner.json


In [605]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json


{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769362576_5e47ceec44",
  "status": "ok",
  "started_epoch": 1769362576.2519376,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 50,
    "stress_evidence_dropout_p": 0.0,
    "stress_evidence_dropout_min_keep": 1,
    "stress_contradictory_p": 0.0,
    "stress_contradictory_max_extra": 0
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3",
    "tool_version": "0.0.0"
  },
  "inputs": {
    "evidence_table":

In [606]:
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"

import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")
print("shape", df.shape)
print(df["status"].value_counts(dropna=False))
for c in ["abstain_reason","fail_reason","context_status","context_evaluated","context_method","audit_notes"]:
    if c in df.columns:
        print("\n==", c, "==")
        print(df[c].fillna("").value_counts().head(20))

shape (50, 123)
status
PASS       33
ABSTAIN    17
Name: count, dtype: int64

== abstain_reason ==
abstain_reason
                       33
CONTEXT_GATE            8
unstable                7
context_nonspecific     2
Name: count, dtype: int64

== fail_reason ==
fail_reason
    50
Name: count, dtype: int64

== context_status ==
context_status
PASS       46
WARN        2
ABSTAIN     2
Name: count, dtype: int64

== context_evaluated ==
context_evaluated
True     43
False     7
Name: count, dtype: int64

== context_method ==
context_method
proxy    43
none      7
Name: count, dtype: int64

== audit_notes ==
audit_notes
                                                                                                                                                                     33
context_gate=hard                                                                                                                                                     8
survival[module]=0.766 < tau=0.80       

In [607]:
import json, pandas as pd

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df = pd.read_csv(p, sep="\t")

def has_keys(s, keys):
    try:
        o = json.loads(str(s))
        return {k: (k in o) for k in keys}
    except Exception:
        return None

keys = ["context_evaluated","context_method","context_status","context_reason","context_confidence","context_notes"]
sample = df[df["abstain_reason"]=="context_missing"].head(5)

for i, r in sample.iterrows():
    ck = has_keys(r.get("claim_json",""), keys)
    print("row", i, "claim_json_has", ck)
    # ついでに中身の該当フィールドだけ見たいなら：
    try:
        o = json.loads(r["claim_json"])
        print({k:o.get(k,None) for k in keys})
    except Exception as e:
        print("json load error:", e)
    print("---")


In [608]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("\n".join(df.columns))

claim_id
entity
direction
context_keys
term_uid
source
term_id
term_name
module_id
module_missing
module_reason
gene_ids
gene_ids_suggest
gene_ids_snippet
gene_ids_n_total
gene_ids_n_in_claim
term_ids
gene_set_hash
context_score
context_evaluated
context_status
context_method
context_confidence
context_reason
select_context_score
select_context_evaluated
select_context_tokens_n
select_context_status_score
select_context_tiebreak
context_signature
context_swap_active
context_gate_mode
context_gate_hit
eligible_context
eligible
term_survival
keep_term
keep_reason
claim_json
preselect_tau_gate
context_score_proxy
select_context_mode
select_context_policy
select_notes
select_diag_n_total
select_diag_n_eligible
select_diag_n_ineligible
select_diag_n_picked
select_diag_blocked_by_module_cap
select_diag_relaxed
select_diag_relax_passes
select_diag_max_per_module
select_diag_ctx_hit_terms
claim_mode
llm_notes
context_notes
context_review_mode
context_ctx_id
context_proxy_method
context_score_p

In [609]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20
report   True 109016
audit    True 147646
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 509
meta     True 8102

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 66)
[report] decisions: {'PASS': 38, 'ABSTAIN': 12}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                r

In [611]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20
report   True 115667
audit    True 154710
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 611
meta     True 8231

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 66)
[report] decisions: {'ABSTAIN': 38, 'PASS': 12}
[report] tau unique: [0.2]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id         

In [612]:
import pandas as pd

base_ours = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.20/audit_log.tsv"
base_swap = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.20/audit_log.tsv"

a = pd.read_csv(base_ours, sep="\t")
b = pd.read_csv(base_swap, sep="\t")

def _ensure_decision(df: pd.DataFrame) -> pd.DataFrame:
    # Prefer explicit "decision"; otherwise fall back to "status" (common in audit logs)
    if "decision" not in df.columns and "status" in df.columns:
        df = df.copy()
        df["decision"] = df["status"]
    return df

a = _ensure_decision(a)
b = _ensure_decision(b)

want = [
    "claim_id",
    "term_id",
    "term_name",
    "decision",
    "abstain_reason",
    "context_score_proxy_u01",
    "context_reason",
    "context_swap_active",
    "context_ctx_id_original",
    "context_ctx_id_effective",
    "context_swap_to",
]

def _pick_existing(df: pd.DataFrame, cols: list[str]) -> list[str]:
    return [c for c in cols if c in df.columns]

keep_a = _pick_existing(a, want)
keep_b = _pick_existing(b, want)

aa = a[keep_a].rename(columns=lambda c: c if c == "claim_id" else f"{c}_ours")
bb = b[keep_b].rename(columns=lambda c: c if c == "claim_id" else f"{c}_swap")

m = aa.merge(bb, on="claim_id", how="inner")

# Find flips (ours vs swap)
d = m[(m.get("decision_ours") != m.get("decision_swap"))].copy()

print("cols_used_ours:", keep_a)
print("cols_used_swap:", keep_b)
print("n_flips:", len(d))

show = [c for c in [
    "claim_id",
    "term_name_ours", "term_name_swap",
    "decision_ours", "decision_swap",
    "abstain_reason_ours", "abstain_reason_swap",
    "context_score_proxy_u01_ours", "context_score_proxy_u01_swap",
    "context_swap_to_swap",
] if c in d.columns]

if len(d) == 0:
    print("No decision flips found between ours and context_swap at this tau.")
else:
    print(d[show].to_string(index=False))


cols_used_ours: ['claim_id', 'term_id', 'term_name', 'decision', 'abstain_reason', 'context_score_proxy_u01', 'context_reason', 'context_swap_active', 'context_ctx_id_original', 'context_ctx_id_effective', 'context_swap_to']
cols_used_swap: ['claim_id', 'term_id', 'term_name', 'decision', 'abstain_reason', 'context_score_proxy_u01', 'context_reason', 'context_swap_active', 'context_ctx_id_original', 'context_ctx_id_effective', 'context_swap_to']
n_flips: 32
      claim_id                           term_name_ours                           term_name_swap decision_ours decision_swap abstain_reason_ours abstain_reason_swap  context_score_proxy_u01_ours  context_score_proxy_u01_swap context_swap_to_swap
c_a109d5f2a3c3                       HALLMARK_APOPTOSIS                       HALLMARK_APOPTOSIS          PASS       ABSTAIN                 NaN        CONTEXT_GATE                      0.685916                      0.685916                 LUAD
c_3de6c9d2aa73       HALLMARK_OXIDATIVE_PHOSPH

In [613]:
import re
import numpy as np
import pandas as pd

def extract_u(s):
    if s is None or (isinstance(s, float) and np.isnan(s)):
        return np.nan
    m = re.search(r"\bu=([0-9]*\.?[0-9]+)", str(s))
    return float(m.group(1)) if m else np.nan

# さっき作った m / d を使うならそのまま
m2 = m.copy()

m2["u_gate_ours"] = m2.get("context_reason_ours", pd.Series([np.nan]*len(m2))).map(extract_u)
m2["u_gate_swap"] = m2.get("context_reason_swap", pd.Series([np.nan]*len(m2))).map(extract_u)

# 反転のみ確認
d2 = m2[m2["decision_ours"] != m2["decision_swap"]].copy()

cols = [
    "claim_id",
    "term_name_ours",
    "decision_ours","decision_swap",
    "abstain_reason_ours","abstain_reason_swap",
    "u_gate_ours","u_gate_swap",
    "context_reason_ours","context_reason_swap",
]
cols = [c for c in cols if c in d2.columns]
print(d2[cols].to_string(index=False))

# 混同行列（PASS/ABSTAIN の移動量）
cm = pd.crosstab(m2["decision_ours"], m2["decision_swap"])
print("\nconfusion:\n", cm)

# net の増減（PASS がどれだけ動いたか）
n_pass_ours = (m2["decision_ours"] == "PASS").sum()
n_pass_swap = (m2["decision_swap"] == "PASS").sum()
print("\nPASS ours:", n_pass_ours, "PASS swap:", n_pass_swap, "delta:", n_pass_swap - n_pass_ours)


      claim_id                           term_name_ours decision_ours decision_swap abstain_reason_ours abstain_reason_swap  u_gate_ours  u_gate_swap context_reason_ours context_reason_swap
c_a109d5f2a3c3                       HALLMARK_APOPTOSIS          PASS       ABSTAIN                 NaN        CONTEXT_GATE          NaN          NaN proxy_context_score proxy_context_score
c_3de6c9d2aa73       HALLMARK_OXIDATIVE_PHOSPHORYLATION          PASS       ABSTAIN                 NaN        CONTEXT_GATE          NaN          NaN proxy_context_score proxy_context_score
c_3e9cb46ac1f2                HALLMARK_MTORC1_SIGNALING          PASS       ABSTAIN                 NaN context_nonspecific          NaN          NaN proxy_context_score proxy_context_score
c_40f168bf3a97           HALLMARK_FATTY_ACID_METABOLISM          PASS       ABSTAIN                 NaN context_nonspecific          NaN          NaN proxy_context_score proxy_context_score
c_f486da562f1b         HALLMARK_PI3K_AKT_MTOR_SIGN

In [614]:
import pandas as pd

def probe_u_cols(df):
    cand = [c for c in df.columns if ("proxy" in c and "u01" in c) or c.endswith("_u01") or "u_gate" in c]
    out = []
    for c in cand:
        s = pd.to_numeric(df[c], errors="coerce")
        out.append((c, float(s.min()), float(s.max()), int(s.notna().sum())))
    return pd.DataFrame(out, columns=["col","min","max","n_non_na"]).sort_values(["n_non_na","col"], ascending=[False,True])

print("OURS")
display(probe_u_cols(a))   # a は ours の audit df
print("SWAP")
display(probe_u_cols(b))   # b は swap の audit df


OURS


,col,min,max,n_non_na
1,context_score_proxy_u01,0.037075,0.958041,50
3,context_score_proxy_u01_from_proposed,0.025939,0.999977,50
2,context_score_proxy_u01_norm,0.025939,0.999977,50
4,context_score_proxy_u01_norm_from_proposed,0.025939,0.999977,50
0,preselect_tau_gate,0.000000,0.000000,50


SWAP


,col,min,max,n_non_na
1,context_score_proxy_u01,0.037075,0.958041,50
3,context_score_proxy_u01_from_proposed,0.026690,0.981552,50
2,context_score_proxy_u01_norm,0.026690,0.981552,50
4,context_score_proxy_u01_norm_from_proposed,0.026690,0.981552,50
0,preselect_tau_gate,0.000000,0.000000,50


In [615]:
p_warn = 0.2

keep = [
    "claim_id","term_name","decision","abstain_reason",
    "context_score_proxy_u01","context_score_proxy_u01_norm",
    "context_ctx_id_effective","context_ctx_id_original",
    "context_swap_to","context_swap_active",
    "context_gate_hit","eligible","status",  # ここが超重要（あれば）
]

aa = a[[c for c in keep if c in a.columns]].rename(columns=lambda c: c+"_ours" if c!="claim_id" else c)
bb = b[[c for c in keep if c in b.columns]].rename(columns=lambda c: c+"_swap" if c!="claim_id" else c)
m = aa.merge(bb, on="claim_id", how="inner")

m["u_ours"] = pd.to_numeric(m.get("context_score_proxy_u01_ours"), errors="coerce")
m["u_swap"] = pd.to_numeric(m.get("context_score_proxy_u01_swap"), errors="coerce")
m["hit_ours"] = m["u_ours"] < p_warn
m["hit_swap"] = m["u_swap"] < p_warn

flips = m[m["decision_ours"] != m["decision_swap"]].copy()
flips["crossed_threshold"] = flips["hit_ours"] != flips["hit_swap"]
flips["delta_u"] = flips["u_swap"] - flips["u_ours"]

show_cols = [
    "claim_id",
    "term_name_ours","term_name_swap",
    "decision_ours","decision_swap",
    "abstain_reason_ours","abstain_reason_swap",
    "u_ours","u_swap","delta_u",
    "hit_ours","hit_swap","crossed_threshold",
    "context_gate_hit_ours","context_gate_hit_swap",
    "eligible_ours","eligible_swap",
    "status_ours","status_swap",
    "context_ctx_id_effective_ours","context_ctx_id_effective_swap",
    "context_swap_to_swap",
]
show_cols = [c for c in show_cols if c in flips.columns]

flips_sorted = flips.sort_values(["crossed_threshold","delta_u"], ascending=[False, True])
display(flips_sorted[show_cols])
print("n_flips:", len(flips_sorted))
print("n_crossed_threshold:", int(flips_sorted["crossed_threshold"].sum()))


,claim_id,term_name_ours,term_name_swap,decision_ours,decision_swap,abstain_reason_ours,abstain_reason_swap,u_ours,u_swap,delta_u,...,crossed_threshold,context_gate_hit_ours,context_gate_hit_swap,eligible_ours,eligible_swap,status_ours,status_swap,context_ctx_id_effective_ours,context_ctx_id_effective_swap,context_swap_to_swap
0,c_a109d5f2a3c3,HALLMARK_APOPTOSIS,HALLMARK_APOPTOSIS,PASS,ABSTAIN,NaN,CONTEXT_GATE,0.685916,0.685916,0.0,...,False,False,False,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
2,c_3de6c9d2aa73,HALLMARK_OXIDATIVE_PHOSPHORYLATION,HALLMARK_OXIDATIVE_PHOSPHORYLATION,PASS,ABSTAIN,NaN,CONTEXT_GATE,0.958041,0.958041,0.0,...,False,False,False,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
5,c_3e9cb46ac1f2,HALLMARK_MTORC1_SIGNALING,HALLMARK_MTORC1_SIGNALING,PASS,ABSTAIN,NaN,context_nonspecific,0.371790,0.371790,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
6,c_40f168bf3a97,HALLMARK_FATTY_ACID_METABOLISM,HALLMARK_FATTY_ACID_METABOLISM,PASS,ABSTAIN,NaN,context_nonspecific,0.183519,0.183519,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
7,c_f486da562f1b,HALLMARK_PI3K_AKT_MTOR_SIGNALING,HALLMARK_PI3K_AKT_MTOR_SIGNALING,ABSTAIN,PASS,CONTEXT_GATE,NaN,0.846726,0.846726,0.0,...,False,False,False,False,False,ABSTAIN,PASS,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
8,c_7132ece5e87e,HALLMARK_E2F_TARGETS,HALLMARK_E2F_TARGETS,PASS,ABSTAIN,NaN,context_nonspecific,0.148689,0.148689,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
9,c_71353c2d3baa,HALLMARK_UV_RESPONSE_UP,HALLMARK_UV_RESPONSE_UP,PASS,ABSTAIN,NaN,context_nonspecific,0.174586,0.174586,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
10,c_715e2eaa872c,HALLMARK_INTERFERON_GAMMA_RESPONSE,HALLMARK_INTERFERON_GAMMA_RESPONSE,PASS,ABSTAIN,NaN,CONTEXT_GATE,0.889209,0.889209,0.0,...,False,False,False,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
11,c_941d2f3967a9,HALLMARK_BILE_ACID_METABOLISM,HALLMARK_BILE_ACID_METABOLISM,PASS,ABSTAIN,NaN,context_nonspecific,0.074254,0.074254,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD
13,c_ba0bf933a576,HALLMARK_ANDROGEN_RESPONSE,HALLMARK_ANDROGEN_RESPONSE,PASS,ABSTAIN,NaN,context_nonspecific,0.491576,0.491576,0.0,...,False,False,True,False,False,PASS,ABSTAIN,HNSC|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD|tumor|genotype|TP53_mut_vs_TP53_wt,LUAD


n_flips: 32
n_crossed_threshold: 0


In [616]:
# swap のみでOK
b[["context_gate_hit","eligible_context","context_status","select_context_status_score","context_score_proxy_u01"]].head(10)
print(b["context_status"].value_counts(dropna=False).head(10))
print(b["select_context_status_score"].value_counts(dropna=False).head(10))
print(b["context_gate_hit"].value_counts(dropna=False))
print(b["eligible_context"].value_counts(dropna=False))


context_status
WARN    33
PASS    17
Name: count, dtype: int64
select_context_status_score
0    33
1    17
Name: count, dtype: int64
context_gate_hit
True     33
False    17
Name: count, dtype: int64
eligible_context
False    33
True     17
Name: count, dtype: int64


In [617]:
!ls /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

audit_log.tsv		    modules.tsv		  sample_card.original.json
claims.proposed.tsv	    report.jsonl	  sample_card.resolved.json
distilled.tsv		    report.md		  sample_card.tau.json
distilled.with_modules.tsv  risk_coverage.tsv	  term_gene_edges.tsv
evidence.normalized.tsv     run_meta.json	  term_modules.tsv
manifest.json		    run_meta.runner.json


In [618]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 109285
audit    True 146709
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 509
meta     True 8102

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 66)
[report] decisions: {'PASS': 33, 'ABSTAIN': 17}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                r

In [619]:
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80/run_meta.json
!cat /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80/sample_card.tau.json


{
  "tool": "llm-pathway-curator",
  "cmd": "run",
  "run_id": "1769362577_79e314cd08",
  "status": "ok",
  "started_epoch": 1769362577.2095938,
  "config": {
    "evidence_table": "/work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv",
    "sample_card": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80/sample_card.tau.json",
    "outdir": "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80",
    "force": true,
    "seed": 42,
    "run_meta_name": "run_meta.json",
    "tau": 0.8,
    "k_claims": 50,
    "stress_evidence_dropout_p": 0.0,
    "stress_evidence_dropout_min_keep": 1,
    "stress_contradictory_p": 0.0,
    "stress_contradictory_max_extra": 0
  },
  "env": {
    "python": "3.12.12 | packaged by conda-forge | (main, Oct 22 2025, 23:16:53) [GCC 14.3.0]",
    "platform": "Linux-6.12.54-linuxkit-aarch64-with-glibc2.39",
    "pandas": "2.3.3",
    "tool_version": "0.0.0"
  },
  "inputs": {
    "

In [620]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/context_swap/gate_hard/tau_0.80
report   True 114951
audit    True 152850
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 611
meta     True 8231

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 66)
[report] decisions: {'ABSTAIN': 39, 'PASS': 11}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id         

In [621]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants stress \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 100 \
  --stress-evidence-dropout-p 0.05 \
  --stress-evidence-dropout-min-keep 1 \
  --stress-contradictory-p 0 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: stress
  gate_modes: hard
  taus: 0.80
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
  stress:
    recipe: dropout
    evidence_dropout_p: 0.050
    evidence_dropout_min_keep: 1
    contradictory_p: 0.000
    contradictory_max_extra: 0
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC stress gate=hard tau=0.80
[run_fig2] WARNING: stress variant ran but pipeline recorded both evidence_dropout_p=0 and contradictory_p=0 (stress may not have been applied). outdir=/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out


In [622]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 109285
audit    True 146709
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 509
meta     True 8102

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 66)
[report] decisions: {'PASS': 33, 'ABSTAIN': 17}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                r

In [623]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80


BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/stress/gate_hard/tau_0.80
report   True 121443
audit    True 153642
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 542
meta     True 8412

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 69)
[report] decisions: {'ABSTAIN': 42, 'PASS': 8}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                

In [589]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*

!LLMPATH_BACKEND=ollama \
LLMPATH_OLLAMA_HOST=http://ollama:11434 \
LLMPATH_OLLAMA_MODEL=llama3.1:8b \
LLMPATH_CONTEXT_REVIEW_MODE=llm \
LLMPATH_CONTEXT_GATE_MODE=hard \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 3 \
  --context-review-mode llm \
  --force

import json
m=json.load(open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json"))
print(m["inputs"]["llm"])

p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df = pd.read_csv(p, sep="\t")
print(df["context_method"].fillna("").value_counts().head(10))
print(df["context_evaluated"].fillna(False).astype(bool).value_counts())

!mv "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv" "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log_LLM.tsv"

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 3
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
[SANITY][WARN] context_review_mode=llm but context_method has 0 llm (proxy=3, none=0). Review likely never called; falling back or skipping everywhere.
[run_fig2] OK
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
{'claim': {'enabled': False, 'claim_mode_env': '', 'backend_env': 'ollama', 'backend_enabled': False, 'backend_notes': ''}, 'review': {'enabled': True, 'review_mode': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}, 'select_entrypoint': {'backend_attached': True, 'backend_role': 'review'}}
context_method
none    3
Name: count, dtype: int64
context_evaluated
False    3
Name: count, dtype: int64


In [590]:
import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log_LLM.tsv"
df=pd.read_csv(p, sep="\t")

print("review_mode:", df["context_review_mode"].value_counts(dropna=False).to_dict())
print("tokens_sum:", df["select_context_tokens_n"].fillna(0).astype(int).sum())
print("tokens_dist:", df["select_context_tokens_n"].fillna(0).astype(int).value_counts().head(10).to_dict())

x = df[df["abstain_reason"].fillna("").eq("CONTEXT_GATE")].copy()

cols = [c for c in [
  "claim_id","decision","abstain_reason",
  "context_gate_mode","context_gate_blocked","context_gate_hit",
  "eligible_context","eligible",
  "context_method","context_evaluated","context_status","context_reason","context_confidence",
  "select_context_mode","select_context_status_score","select_context_score","select_context_tokens_n",
  "context_method_from_proposed","context_status_from_proposed","context_reason_from_proposed","context_confidence_from_proposed"
] if c in x.columns]

print(x[cols].head(30).to_string(index=False))
print("\ncontext_method:", x["context_method"].value_counts(dropna=False).to_dict())
print("context_method_from_proposed:", x["context_method_from_proposed"].value_counts(dropna=False).to_dict())

y = df[df["abstain_reason"].fillna("").eq("CONTEXT_GATE")].copy()
cols = [c for c in ["claim_id","decision","abstain_reason",
                    "context_status_from_proposed","context_reason_from_proposed","context_confidence_from_proposed",
                    "select_context_status_score","select_context_tokens_n"] if c in y.columns]
print(y[cols].head(25).to_string(index=False))


FileNotFoundError: [Errno 2] No such file or directory: '/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log_LLM.tsv'

In [ ]:
import json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/sample_card.tau.json"
c=json.load(open(p))
print("extra.context_review_mode =", c.get("extra",{}).get("context_review_mode"))
print("extra.context_gate_mode   =", c.get("extra",{}).get("context_gate_mode"))

import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print(df["select_context_mode"].value_counts(dropna=False).head())
print(df["select_context_tokens_n"].fillna(0).astype(int).sum())

import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("context_method counts:", df["context_method"].value_counts(dropna=False).to_dict())
print("context_review_mode counts:", df["context_review_mode"].value_counts(dropna=False).to_dict())

import pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")

x=df[df["context_method"].fillna("").astype(str).str.lower().eq("none")].copy()
print("n_none =", len(x))

cols=[c for c in [
  "claim_id","decision","abstain_reason","fail_reason",
  "context_evaluated","context_status","context_reason","context_notes",
  "context_confidence","context_review_mode",
  "select_context_mode","select_context_status_score","select_context_score"
] if c in x.columns]

print(x[cols].head(20).to_string(index=False))
print("\ncontext_evaluated counts (none):", x["context_evaluated"].value_counts(dropna=False).to_dict())
print("context_status counts (none):", x["context_status"].value_counts(dropna=False).to_dict())


In [487]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 107724
audit    True 143857
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 1615
card     True 509
meta     True 7597

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (50, 65)
[report] decisions: {'PASS': 41, 'ABSTAIN': 9}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                ru

In [484]:
!rm -rf /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/*

!LLMPATH_BACKEND="ollama" \
LLMPATH_OLLAMA_HOST="http://ollama:11434" \
LLMPATH_OLLAMA_MODEL="llama3.1:8b" \
LLMPATH_CLAIM_MODE=llm \
python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC \
  --variants ours \
  --gate-modes hard \
  --taus 0.8 \
  --k-claims 50 \
  --context-review-mode llm \
  --force

OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!cat $OUT/llm_claims.raw.json
!cat $OUT/llm_claims.meta.json

import json
m=json.load(open("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/run_meta.json"))
print(m["inputs"]["llm"])

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC
  variants: ours
  gate_modes: hard
  taus: 0.80
  k_claims: 50
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 1
[run_fig2] (1/1) HNSC ours gate=hard tau=0.80
^C
Traceback (most recent call last):
  File "/work/paper/scripts/run_fig2_pipeline.py", line 1065, in <module>
    main()
  File "/work/paper/scripts/run_fig2_pipeline.py", line 1058, in main
    _run_one(job, force=bool(args.force), plan=plan, backend=backend)
  File "/work/paper/scripts/run_fig2_pipeline.py", line 592, in _run_one
    run_pipeline(cfg)
  File "/opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py", line 2832, in run_pipeline
    proposed = _call_compat(
               ^^^^^^^^^^^^^
  File "/opt/LLM-PathwayCurator/src/llm_pathway_curator/pipeline.py", line 2622, in _call_compat
    return fn(*args, **filt)
           ^^^^^^^^^^^^^^^^^
  File "/opt/LLM-PathwayCurator/src/llm_pathway_curator/select.py", line 1

extra.context_review_mode = llm
extra.context_gate_mode   = hard
select_context_mode
off    50
Name: count, dtype: int64
0


In [272]:
import pandas as pd
df=pd.read_csv("/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv", sep="\t")
print([c for c in df.columns if c.startswith("context")])

['context_key', 'context_keys', 'context_score', 'context_evaluated', 'context_method', 'context_status', 'context_reason', 'context_notes', 'context_confidence', 'context_gate_mode', 'context_review_mode', 'context_ctx_id', 'context_proxy_method', 'context_score_proxy', 'context_score_proxy_norm', 'context_tokens_n', 'context_signature', 'context_score_proxy_u01', 'context_score_proxy_u01_norm', 'context_ctx_id_original', 'context_ctx_id_effective', 'context_swap_from', 'context_swap_to']


In [273]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

!ls -lh $OUT/{run_meta.json,claims.proposed.tsv,audit_log.tsv,report.jsonl} 2>/dev/null || true
!wc -l  $OUT/claims.proposed.tsv  $OUT/audit_log.tsv  $OUT/report.jsonl 2>/dev/null || true

import json, os
p=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")+"/run_meta.json"
m=json.load(open(p))
print("step:", m.get("step"), "status:", m.get("status"))
print("llm:", m.get("inputs",{}).get("llm",{}))
print("claims:", m.get("inputs",{}).get("claims",{}))

   13 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv
   13 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv
   12 /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl
   38 total
step: report_jsonl status: ok
llm: {'claim': {'enabled': True, 'claim_mode_env': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}, 'review': {'enabled': True, 'review_mode': 'llm', 'backend_env': 'ollama', 'backend_enabled': True, 'backend_notes': ''}, 'select_entrypoint': {'backend_attached': True, 'backend_role': 'claim'}}
claims: {'context_gate_review_compat': {'compat_action': 'none', 'gate_mode': 'hard', 'review_mode': 'llm'}, 'k_source': 'cfg', 'k_cfg': 50, 'k_env': '', 'k_card': '', 'k_effective': 50, 'k_env_raw': '', 'context_gate_mode': 'hard', 'context_review_mode': 'llm', 'context_ctx_id_original': 'HNSC|tumor|genotype|TP53_mut_vs_TP53_wt', 'context_ctx_id_ef

In [274]:
!python paper/scripts/fig2_sanity_check.py /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80

BASE: /work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80
report   True 27200
audit    True 41779
modules  True 14057
edges    True 88967
terms    True 3188
distilled True 40803
risk     True 604
card     True 509
meta     True 7544

[run_meta] tool/status/step: llm-pathway-curator ok report_jsonl

[report] shape: (12, 65)
[report] decisions: {'PASS': 9, 'ABSTAIN': 3}
[report] tau unique: [0.8]
[report] columns (top 25): ['created_at', 'run_id', 'method', 'condition', 'disease', 'cancer', 'comparison', 'tau', 'species', 'claim_id', 'decision_status', 'decision', 'survival', 'claim.entity_type', 'claim.entity_id', 'claim.context.condition', 'claim.context.tissue', 'claim.context.perturbation', 'claim.context.comparison', 'claim.context.disease', 'evidence_refs.gene_set_hash', 'context_evaluated', 'context_method', 'context_status', 'context_reason']

[report] contract view (head):
[SANITY][WARN] missing columns: ['schema_version']
  benchmark_id                run_id

In [64]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
p=f"{out}/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
cols=[c for c in df.columns if c.startswith("context_review_")]
print("context_review_* cols:", cols)
print(df[cols].head(3) if cols else "(none)")

import pandas as pd, os, json
out=os.environ.get("OUT","/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80")
df=pd.read_csv(f"{out}/claims.proposed.tsv", sep="\t")
cj=df.loc[0,"claim_json"]
obj=json.loads(cj)
keys=[k for k in obj.keys() if k.startswith("context_")]
print("claim_json context_* keys:", keys)
print({k: obj.get(k) for k in keys})

context_review_* cols: ['context_review_evaluated', 'context_review_status', 'context_review_reason', 'context_review_notes', 'context_review_confidence', 'context_review_mode', 'context_review_method']
   context_review_evaluated context_review_status context_review_reason  \
0                      True                  PASS  consistent_direction   
1                      True                  PASS                   CPT   
2                      True                  PASS                     C   

                                context_review_notes  \
0  Direction consistent with evidence genes in TP...   
1  TP53 pathway is relevant in HNSC with TP53 mut...   
2  TNFA signaling pathway is relevant to HNSC tum...   

   context_review_confidence context_review_mode context_review_method  
0                        0.8                 llm                   llm  
1                        1.0                 llm                   llm  
2                        0.8                 llm    

In [65]:
import json, pandas as pd
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
row=df.iloc[0]
cj=json.loads(row["claim_json"])
print("context_keys:", cj.get("context_keys"))

context_keys: ['condition', 'tissue', 'perturbation', 'comparison']


In [66]:
import pandas as pd
p="paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/audit_log.tsv"
df=pd.read_csv(p, sep="\t")
print("cols:", [c for c in df.columns if "claim_json" in c])
print("has_claim_json:", "claim_json" in df.columns)
print("has_claim_json_str:", "claim_json_str" in df.columns)

cols: ['claim_json']
has_claim_json: True
has_claim_json_str: False


In [67]:
OUT="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80"
!ls -lh "$OUT" | sed -n '1,200p'

for f in ["distilled.tsv", "distilled_with_modules.tsv modules.tsv", "evidence.normalized.tsv", "evidence.tsv",]:
  ![ -f "$OUT/$f" ] && echo "== $f ==" && wc -l "$OUT/$f" && head -n 3 "$OUT/$f"



total 444K
-rw-r--r-- 1 root root  37K Jan 24 01:28 audit_log.tsv
-rw-r--r-- 1 root root  25K Jan 24 01:28 claims.proposed.tsv
-rw-r--r-- 1 root root 2.6K Jan 24 01:28 context_review_cache.jsonl
-rw-r--r-- 1 root root  40K Jan 24 01:28 distilled.tsv
-rw-r--r-- 1 root root  40K Jan 24 01:17 distilled.with_modules.tsv
-rw-r--r-- 1 root root  15K Jan 24 01:17 evidence.normalized.tsv
-rw-r--r-- 1 root root  43K Jan 24 01:25 llm_claims.candidates.json
-rw-r--r-- 1 root root  378 Jan 24 01:25 llm_claims.meta.json
-rw-r--r-- 1 root root 3.3K Jan 24 01:25 llm_claims.prompt.json
-rw-r--r-- 1 root root 2.8K Jan 24 01:25 llm_claims.raw.json
-rw-r--r-- 1 root root 2.3K Jan 24 01:28 manifest.json
-rw-r--r-- 1 root root  14K Jan 24 01:17 modules.tsv
-rw-r--r-- 1 root root  25K Jan 24 01:28 report.jsonl
-rw-r--r-- 1 root root  56K Jan 24 01:28 report.md
-rw-r--r-- 1 root root   53 Jan 24 01:28 risk_coverage.tsv
-rw-r--r-- 1 root root 6.3K Jan 24 01:28 run_meta.json
-rw-r--r-- 1 root root  653 Jan 24 

In [68]:
import pandas as pd
p = "/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/distilled.tsv"
df = pd.read_csv(p, sep="\t")
print(df.shape)
for c in ["keep_term","eligible","term_survival","keep_reason"]:
    if c in df.columns:
        print(c, df[c].value_counts(dropna=False).head(10))

(50, 54)
keep_term keep_term
True    50
Name: count, dtype: int64
term_survival term_survival
0.984375    10
1.000000     7
0.968750     7
0.953125     5
0.921875     4
0.906250     3
0.937500     3
0.750000     2
0.843750     2
0.875000     1
Name: count, dtype: int64
keep_reason keep_reason
ok    50
Name: count, dtype: int64


In [69]:
import pandas as pd, json
p="/work/paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/claims.proposed.tsv"
df=pd.read_csv(p, sep="\t")
print("rows:", df.shape[0])
print("context_confidence unique:", df.get("context_confidence").unique() if "context_confidence" in df.columns else None)
print("context_review_confidence unique:", df.get("context_review_confidence").unique() if "context_review_confidence" in df.columns else None)


rows: 12
context_confidence unique: [0.8 1. ]
context_review_confidence unique: [0.8 1. ]


In [70]:
# context未評価でABSTAINになっているか
!grep -n '"decision_status": "ABSTAIN"' \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head

# audit_notes に context_missing(hard) が残っているか
!grep -n "context_missing(hard)" \
  paper/source_data/PANCAN_TP53_v1/out/HNSC/ours/gate_hard/tau_0.80/report.jsonl | head


In [71]:
# 1) DNS/ネットワーク
!getent hosts ollama || ping -c 1 ollama

# 2) Ollama API 生存確認
!curl -sS http://ollama:11434/api/tags | head

# 3) 推論が通るか（最小）
!curl -sS http://ollama:11434/api/generate \
  -H 'Content-Type: application/json' \
  -d '{"model":"llama3.1:8b","prompt":"Say OK","stream":false}' | head


172.18.0.2      ollama
{"models":[{"name":"llama3.1:8b","model":"llama3.1:8b","modified_at":"2026-01-01T16:13:01.415171001Z","size":4920753328,"digest":"46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e","details":{"parent_model":"","format":"gguf","family":"llama","families":["llama"],"parameter_size":"8.0B","quantization_level":"Q4_K_M"}}]}{"model":"llama3.1:8b","created_at":"2026-01-24T01:28:44.129203049Z","response":"OK. What's next?","done":true,"done_reason":"stop","context":[128006,882,128007,271,46864,10619,128009,128006,78191,128007,271,4012,13,3639,596,1828,30],"total_duration":1765110501,"load_duration":86553125,"prompt_eval_count":12,"prompt_eval_duration":768796917,"eval_count":7,"eval_duration":904251668}

In [ ]:
import os, json, requests
host = os.environ.get("LLMPATH_OLLAMA_HOST","http://ollama:11434")
model = os.environ.get("LLMPATH_OLLAMA_MODEL","llama3.1:8b")
payload = {"model": model, "prompt": "Say OK", "stream": False}
r = requests.post(f"{host}/api/generate", json=payload, timeout=120)
print("status", r.status_code)
print("text", r.text[:200])
print("json", r.json())

In [ ]:
import inspect
import llm_pathway_curator.backends as b
print("backends.py:", b.__file__)
print("--- OllamaBackend.generate source ---")
print(inspect.getsource(b.OllamaBackend.generate))

In [ ]:
from llm_pathway_curator.backends import get_backend_from_env
b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, getattr(b, "host", None), getattr(b, "model_name", None))

s = b.generate('{"ping": "pong"}', json_mode=True)
print("returned:", s[:200])

In [ ]:
!grep -RIn "backend_call_failed" -n src/llm_pathway_curator
!grep -RIn "llm_claims.raw.json" -n src/llm_pathway_curator
!grep -RIn "llm_claims.meta.json" -n src/llm_pathway_curator


In [ ]:
import json
from llm_pathway_curator.backends import get_backend_from_env

b = get_backend_from_env(seed=42)
print("backend:", type(b).__name__, b.host, b.model_name)

# 1) json_mode で “スキーマっぽい” JSONを返させる
prompt = '{"claims": [{"claim_id":"c1","entity":"X","direction":"up","evidence_refs":["m1"],"context":{"disease":"HNSC"} }]}'
out = b.generate(prompt, json_mode=True)
print("raw_out_head:", out[:200])

# 2) ちゃんとJSONとして読めるか
obj = json.loads(out)
print("top_keys:", list(obj.keys())[:10])

In [77]:
for c in ("HNSC", "LUAD", "LUSC", "BRCA", "OV",  "UCEC",  "SKCM"):
    !Rscript paper/scripts/fig2_fgsea_to_evidence_table.R $c

[fgsea_to_evidence] inputs
  cancer: HNSC 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived/rankings/HNSC.deg_ranking.tsv 
  out : /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  msigdbr_version: 25.1.1 
  fgsea_version  : 1.36.2 
[fgsea_to_evidence] stats diagnostics
  stats_len: 20262 
  stats_type: double 
  stats_na: 0 
  stats_range: -12.6721379948078,10.5253182402821 
  gene_example: 4060,6817,5153,1902,19594,9055,6447,16500,1970,8301 
  gene_id_type: entrez
[fgsea_to_evidence] msigdbr entrez column: ncbi_gene 
[fgsea_to_evidence] overlap diagnostics
  overlap_summary: min 24 p25 80 median 141 p75 165 max 193 
  pathways_with_overlap>=15: 50 / 50 
[fgsea_to_evidence] fgsea diagnostics
  n_pathways: 50 
  res_rows  : 50 
[fgsea_to_evidence] OK
  wrote: /work/paper/source_data/PANCAN_TP53_v1/evidence_tables/HNSC.evidence_table.tsv 
  n_terms: 50 
[fgsea_to_evidence] inputs
  cancer: LUAD 
  rank: /work/paper/source_data/PANCAN_TP53_v1/derived

In [142]:
!python paper/scripts/run_fig2_pipeline.py \
  --cancers HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM \
  --variants ours,shuffled_context \
  --gate-modes hard \
  --taus 0.2,0.4,0.6,0.8,0.9 \
  --force

[run_fig2] plan
  benchmark_id: PANCAN_TP53_v1
  cancers: HNSC,LUAD,LUSC,BRCA,OV,UCEC,SKCM
  variants: ours,shuffled_context
  gate_modes: hard
  taus: 0.20,0.40,0.60,0.80,0.90
  k_claims: 100
  seed: 42
  out_root: /work/paper/source_data/PANCAN_TP53_v1/out
[run_fig2] jobs: 70
[run_fig2] (1/70) HNSC ours gate=hard tau=0.20
[run_fig2] (2/70) HNSC shuffled_context gate=hard tau=0.20
[run_fig2] (3/70) HNSC ours gate=hard tau=0.40
[run_fig2] (4/70) HNSC shuffled_context gate=hard tau=0.40
[run_fig2] (5/70) HNSC ours gate=hard tau=0.60
[run_fig2] (6/70) HNSC shuffled_context gate=hard tau=0.60
[run_fig2] (7/70) HNSC ours gate=hard tau=0.80
[run_fig2] (8/70) HNSC shuffled_context gate=hard tau=0.80
[run_fig2] (9/70) HNSC ours gate=hard tau=0.90
[run_fig2] (10/70) HNSC shuffled_context gate=hard tau=0.90
[run_fig2] (11/70) LUAD ours gate=hard tau=0.20
[run_fig2] (12/70) LUAD shuffled_context gate=hard tau=0.20
[run_fig2] (13/70) LUAD ours gate=hard tau=0.40
[run_fig2] (14/70) LUAD shuffled_c

In [143]:
!python paper/scripts/fig2_make_labels_template.py \
  --root paper/source_data/PANCAN_TP53_v1/out \
  --out  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --per-run 20 \
  --prefer-status PASS \
  --with-hints

[OK] wrote: paper/source_data/PANCAN_TP53_v1/labels.tsv rows=150
[NEXT] Fill human_label with: ACCEPT / REJECT / SHOULD_ABSTAIN (partial labels ok)


In [141]:
!python paper/scripts/fig2_collect_risk_coverage.py \
  --root /work/paper/source_data/PANCAN_TP53_v1/out \
  --out  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --benchmark-id PANCAN_TP53_v1


[OK] wrote: /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv rows=70
[WARN] n_pass_labeled is 0 for all rows. Use risk_proxy or provide --labels.


In [144]:
!python paper/scripts/bench_fig2.py \
  --reports paper/source_data/PANCAN_TP53_v1/out \
  --labels  paper/source_data/PANCAN_TP53_v1/labels.tsv \
  --out     /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --taus    0:1:0.05 \
  --rater-id R1


Traceback (most recent call last):
  File "/work/paper/scripts/bench_fig2.py", line 315, in <module>
    main()
  File "/work/paper/scripts/bench_fig2.py", line 303, in main
    reports_df = load_reports(report_paths)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/paper/scripts/bench_fig2.py", line 123, in load_reports
    raise ValueError(
ValueError: Missing metrics.survival in paper/source_data/PANCAN_TP53_v1/out/BRCA/ours/gate_hard/tau_0.20/report.jsonl line 1 (claim_id=c_a1ed09ef8099)


In [104]:
!python paper/scripts/fig2_plot_multipanel.py \
  --in  /work/paper/source_data/PANCAN_TP53_v1/out/fig2_risk_coverage.tsv \
  --out paper/source_data/PANCAN_TP53_v1/fig/Fig2_Supp_multipanel_proxy.pdf \
  --ycol risk_proxy \
  --benchmark-id PANCAN_TP53_v1 \
  --cols 4 \
  --fontsize 16 \
  --title "Risk–coverage curves across cancers"
